## Initialization

In [1]:
!git clone https://github.com/theophile-bb/Pyspark-stock-analysis.git

Cloning into 'Pyspark-stock-analysis'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 58 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 263.38 KiB | 4.79 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

!pip install -q pyspark findspark

In [3]:
%cd Pyspark-stock-analysis

/content/Pyspark-stock-analysis


In [4]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.2/593.2 kB 12.6 MB/s eta 0:00:00


In [5]:
import findspark
findspark.init()

In [6]:
import sys
sys.path.append("src")

from utils import *

#spark = create_spark_session()

***

## Exploration

Loading all our datasets

In [7]:
folder = "Data/"

file1 = "AMAZON.csv"
file2 = "APPLE.csv"
file3 = "ZOOM.csv"
file4 = "TESLA.csv"
file5 = "FACEBOOK.csv"
file6 = "GOOGLE.csv"
file7 = "MICROSOFT.csv"

dataset = read_file(folder, file1)
dataset2 = read_file(folder, file2)
dataset3 = read_file(folder, file3)
dataset4 = read_file(folder, file4)
dataset5 = read_file(folder, file5)
dataset6 = read_file(folder, file6)
dataset7 = read_file(folder, file7)

In [8]:
get_info_df(dataset3,file3)

Infos of the ZOOM.csv file 


► Schema of the dataset
root
 |-- Date: date (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)

None

 ► Top rows of the dataset
+----------+-----------------+------------------+-----------------+-----------------+--------+-----------------+------------+
|      Date|             High|               Low|             Open|            Close|  Volume|        Adj Close|company_name|
+----------+-----------------+------------------+-----------------+-----------------+--------+-----------------+------------+
|2019-04-18|             66.0| 60.32099914550781|             65.0|             62.0|25764700|             62.0|        ZOOM|
|2019-04-22| 68.9000015258789|59.939998626708984|             61.0|65.69999694824219| 9949700|65.6999

In [9]:
get_info_df(dataset,file1)

Infos of the AMAZON.csv file 


► Schema of the dataset
root
 |-- Date: date (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)

None

 ► Top rows of the dataset
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+
|      Date|             High|              Low|             Open|            Close| Volume|        Adj Close|company_name|
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100|753.6699829101562|      AMAZON|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500|757.17999267578

Average for opening and closing by year, month and week

**Year**

In [10]:
col1 ="Open"
col2 ="Close"

open_year = get_average_year(dataset, col1)
close_year = get_average_year(dataset, col2)

open_year.show(10)
close_year.show(10)

+----+------------------+
|year|      average_Open|
+----+------------------+
|2017|  968.275618959708|
|2018|1644.0727091633466|
|2019|1788.7461896623884|
|2020|2636.5054538710433|
+----+------------------+

+----+------------------+
|year|     average_Close|
+----+------------------+
|2017| 968.1670116409363|
|2018|1641.7261758629545|
|2019| 1789.189206077939|
|2020| 2636.649604240712|
+----+------------------+



**Month**

In [11]:
col1 ="Open"
col2 ="Close"

open_month = get_average_month(dataset, col1)
close_month = get_average_month(dataset, col2)

open_month.show(10)
close_month.show(10)

+----+-----+------------------+
|year|month|      average_Open|
+----+-----+------------------+
|2017|    1|  804.972998046875|
|2017|    2|  833.583162006579|
|2017|    3| 853.0417400857676|
|2017|    4| 904.8905286287006|
|2017|    5|  960.274092240767|
|2017|    6| 995.0422723943537|
|2017|    7|1007.6545013427734|
|2017|    8|  973.247826617697|
|2017|    9| 971.9529998779296|
|2017|   10| 999.5400057705966|
+----+-----+------------------+
only showing top 10 rows

+----+-----+------------------+
|year|month|     average_Close|
+----+-----+------------------+
|2017|    1| 807.5050018310546|
|2017|    2| 835.7452681692023|
|2017|    3| 854.2400008491848|
|2017|    4| 903.3857903731497|
|2017|    5| 961.7195462313565|
|2017|    6| 990.4440890225497|
|2017|    7|1008.4839965820313|
|2017|    8|    971.4365234375|
|2017|    9| 968.9924987792969|
|2017|   10|1000.7199956720525|
+----+-----+------------------+
only showing top 10 rows



**Week**

In [12]:
col1 ="Open"
col2 ="Close"

open_week = get_average_week(dataset, col1)
close_week = get_average_week(dataset, col2)

open_week.show(10)
close_week.show(10)

+----+----+-----------------+
|year|week|     average_Open|
+----+----+-----------------+
|2017|   1|765.0549926757812|
|2017|   2|800.5779907226563|
|2017|   3|812.6200103759766|
|2017|   4|825.8239990234375|
|2017|   5|  825.85400390625|
|2017|   6| 815.443994140625|
|2017|   7|837.2920043945312|
|2017|   8|852.0125122070312|
|2017|   9|849.4320068359375|
|2017|  10|   849.3419921875|
+----+----+-----------------+
only showing top 10 rows

+----+----+-----------------+
|year|week|    average_Close|
+----+----+-----------------+
|2017|   1|771.8224945068359|
|2017|   2|804.5240112304688|
|2017|   3|808.6424865722656|
|2017|   4|830.3520141601563|
|2017|   5|827.2719970703125|
|2017|   6|817.7340087890625|
|2017|   7|    840.966015625|
|2017|   8|852.3699951171875|
|2017|   9|849.1099975585937|
|2017|  10|849.7180053710938|
+----+----+-----------------+
only showing top 10 rows



Daily and monthly stock variation

**Daily**

In [13]:
stock_diff = daily_difference(dataset)
stock_diff.show(10)

+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|      Date|             High|              Low|             Open|            Close| Volume|        Adj Close|company_name| stock_difference|
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100|753.6699829101562|      AMAZON|             NULL|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500|757.1799926757812|      AMAZON|   3.510009765625|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100|780.4500122070312|      AMAZON|   23.27001953125|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200| 795.989990234375|      AMAZON|15.53997802734375|
|2017-

**Monthly**

In [14]:
stock_diff_month = monthly_difference(dataset)
stock_diff_month.show(10)

+----------+------------------+-----------------+-----------------+-----------------+-------+-----------------+------------+------------------+
|      Date|              High|              Low|             Open|            Close| Volume|        Adj Close|company_name|  stock_difference|
+----------+------------------+-----------------+-----------------+-----------------+-------+-----------------+------------+------------------+
|2017-01-03|  758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100|753.6699829101562|      AMAZON|              NULL|
|2017-02-01|  833.780029296875|824.9400024414062|829.2100219726562|832.3499755859375|3850200|832.3499755859375|      AMAZON| 78.67999267578125|
|2017-03-01| 854.8300170898438| 849.010009765625|853.0499877929688|853.0800170898438|2760100|853.0800170898438|      AMAZON| 20.73004150390625|
|2017-04-03|  893.489990234375|885.4199829101562|            888.0| 891.510009765625|3422300| 891.510009765625|      AMAZON| 38.42999267

Calculating the daily return

In [15]:
updateddf = calculate_daily_return(dataset)
updateddf.show(10)

+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|      Date|             High|              Low|             Open|            Close| Volume|        Adj Close|company_name|     Daily_return|
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100|753.6699829101562|      AMAZON|            -4.25|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500|757.1799926757812|      AMAZON|-1.21002197265625|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100|780.4500122070312|      AMAZON| 18.9000244140625|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200| 795.989990234375|      AMAZON| 13.6300048828125|
|2017-

Average daily return by period

In [16]:
column_name = "Daily_return"

daily_return_year = get_average_year(updateddf, column_name)
daily_return_year.show(10)

daily_return_month = get_average_month(updateddf, column_name)
daily_return_month.show(10)

daily_return_week = get_average_week(updateddf, column_name)
daily_return_week.show(10)

+----+--------------------+
|year|average_Daily_return|
+----+--------------------+
|2017|-0.10860731877178785|
|2018| -2.3465333003921813|
|2019| 0.44301641555059523|
|2020|  0.1441503696687232|
+----+--------------------+

+----+-----+--------------------+
|year|month|average_Daily_return|
+----+-----+--------------------+
|2017|    1|  2.5320037841796874|
|2017|    2|  2.1621061626233553|
|2017|    3|  1.1982607634171196|
|2017|    4|  -1.504738255550987|
|2017|    5|  1.4454539905894885|
|2017|    6| -4.5981833718039775|
|2017|    7|  0.8294952392578125|
|2017|    8|  -1.811303180197011|
|2017|    9| -2.9605010986328124|
|2017|   10|  1.1799899014559658|
+----+-----+--------------------+
only showing top 10 rows

+----+----+--------------------+
|year|week|average_Daily_return|
+----+----+--------------------+
|2017|   1|  6.7675018310546875|
|2017|   2|     3.9460205078125|
|2017|   3| -3.9775238037109375|
|2017|   4|    4.52801513671875|
|2017|   5|     1.4179931640625|
|2017|   

Get the highest daily return either globaly or for a given period

In [17]:
start = "2017-01-03"
end = "2017-01-17"
sorteddf = get_highest_dr(updateddf, start, end)
sorteddf.show(10)

+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|      Date|             High|              Low|             Open|            Close| Volume|        Adj Close|company_name|     Daily_return|
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100|780.4500122070312|      AMAZON| 18.9000244140625|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200| 795.989990234375|      AMAZON| 13.6300048828125|
|2017-01-12|814.1300048828125|            799.5|800.3099975585938|813.6400146484375|4873900|813.6400146484375|      AMAZON|13.33001708984375|
|2017-01-11|            799.5| 789.510009765625|793.6599731445312|  799.02001953125|2992800|  799.02001953125|      AMAZON| 5.36004638671875|
|2017-

**Moving Average**

In [18]:
column = "Close"
n = 5
reference_date="2019-05-03"

moving = calculate_moving_average(dataset,column,n,reference_date)
moving.show(10)

+----------+------------------+------------------+------------------+------------------+-------+------------------+------------+------------------+
|      Date|              High|               Low|              Open|             Close| Volume|         Adj Close|company_name|  Close_moving_avg|
+----------+------------------+------------------+------------------+------------------+-------+------------------+------------+------------------+
|2019-05-03|1964.4000244140625|            1936.0|            1949.0|   1962.4599609375|6381600|   1962.4599609375|      AMAZON|           1927.95|
|2019-05-06|            1959.0|            1910.5|  1917.97998046875| 1950.550048828125|5417800| 1950.550048828125|      AMAZON|1930.3739990234376|
|2019-05-07|1949.0999755859375|1903.3800048828125| 1939.989990234375|            1921.0|5902100|            1921.0|      AMAZON|1929.2699951171876|
|2019-05-08|1935.3699951171875|            1910.0|1918.8699951171875|  1917.77001953125|4078600|  1917.770019531

**Rate of Return (RoR)**

RoR one day to another

RoR on a given time period

In [19]:
start_date = '2018-05-01'
period = 90
ror_df = get_ror_period(dataset, start_date, period)

ror_df.orderBy(F.desc("return_rate")).show()

+----------+------------------+------------------+------------------+------------------+-------+------------------+------------+-------------------+
|      date|              High|               Low|              Open|             Close| Volume|         Adj Close|company_name|        return_rate|
+----------+------------------+------------------+------------------+------------------+-------+------------------+------------+-------------------+
|2018-07-25|1863.8399658203125|1822.6400146484375| 1829.300048828125|1863.6099853515625|3738200|1863.6099853515625|      AMAZON| 0.1778152603550999|
|2018-07-17|  1851.68994140625|1797.3800048828125|  1811.56005859375|1843.9300537109375|5682900|1843.9300537109375|      AMAZON|0.16537739836076173|
|2018-07-18|1858.8800048828125|  1831.27001953125|            1848.0|1842.9200439453125|4861900|1842.9200439453125|      AMAZON| 0.1647390647370897|
|2018-07-24|            1840.0|1809.3800048828125| 1829.010009765625| 1829.239990234375|4278700| 1829.2399

**Correlation between stocks**

In [20]:
dataset2.show()

+----------+------------------+------------------+------------------+------------------+----------+------------------+------------+
|      Date|              High|               Low|              Open|             Close|    Volume|         Adj Close|company_name|
+----------+------------------+------------------+------------------+------------------+----------+------------------+------------+
|2017-01-03|29.082500457763672|28.690000534057617|28.950000762939453|29.037500381469727|1.151276E8|27.277639389038086|       APPLE|
|2017-01-04|29.127500534057617|           28.9375|28.962499618530273|  29.0049991607666| 8.44724E7|27.247108459472656|       APPLE|
|2017-01-05| 29.21500015258789|28.952499389648438|28.979999542236328| 29.15250015258789| 8.87744E7| 27.38566780090332|       APPLE|
|2017-01-06|29.540000915527344| 29.11750030517578| 29.19499969482422|29.477500915527344|1.270076E8| 27.69097137451172|       APPLE|
|2017-01-09|29.857500076293945|29.485000610351562|29.487499237060547|29.7474

In [21]:
start_date="2017-01-10"
end_date="2017-02-02"

correlation_value = calculate_stock_correlation(dataset, dataset2)

print(f"Correlation coefficient between stocks on the given span: {correlation_value}")


Correlation coefficient between stocks on the given span: 0.9292860084204524


***

In [22]:
dataset = get_trading_value(dataset)
dataset2 = get_trading_value(dataset2)
dataset3 = get_trading_value(dataset3)
dataset4 = get_trading_value(dataset4)
dataset5 = get_trading_value(dataset5)
dataset6 = get_trading_value(dataset6)
dataset7 = get_trading_value(dataset7)

In [23]:
fig = plot_stock('Total trading value over period')
col = 'trading_value'

add_trace_plot(dataset, col, fig, file1)
add_trace_plot(dataset2, col, fig, file2)
add_trace_plot(dataset3, col, fig, file3)
add_trace_plot(dataset4, col, fig, file4)
add_trace_plot(dataset5, col, fig, file5)
add_trace_plot(dataset6, col, fig, file6)
add_trace_plot(dataset7, col, fig, file7)


***